In [1]:
!pip install langchain
!pip install langchain-huggingface
!pip install pypdf
!pip install faiss-gpu
!pip install ctransformers
!pip install langchain_community
!pip install sentence_transformers
!pip install python-box

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00


In [2]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
import box
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

In [3]:
model_file="/content/drive/MyDrive/llama-2-7b-chat.ggmlv3.q8_0.bin"

In [4]:
dir=DirectoryLoader("/content/",glob="**/*.pdf",loader_cls=PyPDFLoader)

In [5]:
dir_load=dir.load()

In [6]:
dir_load


[Document(metadata={'source': '/content/drive/MyDrive/10_Banking_Questions_and_Answers.pdf', 'page': 0}, page_content='10 Banking Questions and Answers\nQ1: What is a savings account?\nA: A deposit account to save money and earn interest.\nQ2: What is a current account?\nA: An account for frequent transactions, mainly for businesses.\nQ3: What is a fixed deposit?\nA: A savings instrument that locks funds for a fixed period at a higher interest rate.\nQ4: What is net banking?\nA: Online banking services to manage accounts and transactions.\nQ5: What is a bank overdraft?\nA: A facility allowing withdrawal of more than the account balance.\nQ6: What is a check?\nA: A document instructing a bank to pay a specified amount from an account.\nQ7: What is a credit card?\nA: A card that allows borrowing up to a pre-approved limit.\nQ8: What is a debit card?\nA: A card used to spend directly from a linked bank account.\nQ9: What is a loan?\nA: Money borrowed from a bank, to be repaid with interes

In [7]:
splitter_text=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)

In [8]:
docs= splitter_text.split_documents(dir_load)

In [9]:
docs

[Document(metadata={'source': '/content/drive/MyDrive/10_Banking_Questions_and_Answers.pdf', 'page': 0}, page_content='10 Banking Questions and Answers\nQ1: What is a savings account?\nA: A deposit account to save money and earn interest.\nQ2: What is a current account?\nA: An account for frequent transactions, mainly for businesses.\nQ3: What is a fixed deposit?\nA: A savings instrument that locks funds for a fixed period at a higher interest rate.\nQ4: What is net banking?\nA: Online banking services to manage accounts and transactions.\nQ5: What is a bank overdraft?\nA: A facility allowing withdrawal of more than the account balance.\nQ6: What is a check?\nA: A document instructing a bank to pay a specified amount from an account.\nQ7: What is a credit card?\nA: A card that allows borrowing up to a pre-approved limit.\nQ8: What is a debit card?\nA: A card used to spend directly from a linked bank account.\nQ9: What is a loan?\nA: Money borrowed from a bank, to be repaid with interes

#EMBEDDING

In [10]:
embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device":"cpu"})

<ipython-input-10-85c90188bba8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device":"cpu"})
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugg

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
#embed the document and store in vector db

vector=FAISS.from_documents(docs,embedding)

In [12]:
vector.save_local("db/faiss")

In [13]:
#setting up model
llm=CTransformers(
    model=model_file,
    model_type="llama",
    config={"max_new_tokens":1000,"temperature":0.01}
)

In [14]:
#making prompt template

prompt="""
You are a bank chatbox, provide information only based on the context

Answer only for bank related queries.Suppose the question is out of context
respond out of context.

Context:{context}
Question:{question}
"""

In [15]:
def sq_prompt():
  prompt_template=PromptTemplate(template=prompt,input_variables=["context","question"])
  return prompt_template

In [16]:
def answer(llm_mode_type,prompt_type,v_db):
  ques_ans=RetrievalQA.from_chain_type(
      llm=llm_mode_type,
      chain_type="stuff",
      chain_type_kwargs={"prompt":prompt_type},
      retriever=v_db.as_retriever(search_kwargs={"k":2})

  )
  return ques_ans

In [17]:
def setup_model():
  embbed=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device":"cpu"})
  vdb=FAISS.load_local("db/faiss",embbed,allow_dangerous_deserialization=True)
  pmt=sq_prompt()
  get_ans=answer(llm,pmt,vdb)
  return get_ans

#final load run

In [19]:

llm_qa_model=setup_model()

In [20]:
llm_qa_model

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are a bank chatbox, provide information only based on the context\n\nAnswer only for bank related queries.Suppose the question is out of context\nrespond out of context.\n\nContext:{context}\nQuestion:{question}\n'), llm=CTransformers(client=<ctransformers.llm.LLM object at 0x7c27c03659c0>, model='/content/drive/MyDrive/llama-2-7b-chat.ggmlv3.q8_0.bin', model_type='llama', config={'max_new_tokens': 1000, 'temperature': 0.01}), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores

In [21]:
asking_questions="What is a current account?"
llm_qa_model({"query":asking_questions})

<ipython-input-21-721951153e67>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm_qa_model({"query":asking_questions})


{'query': 'What is a current account?',
 'result': 'Answer: A deposit account for frequent transactions, mainly for businesses.'}

In [22]:
asking_questions=["What is a savings account?",
"What is a debit card?",
"What is a credit card?",
"What is mobile banking?"]

for question in asking_questions:
  response=llm_qa_model({"query":question})
   #Extract the answer (assuming `result` contains the actual answer)
  answer = response.get("result", "No answer provided")


    # Clean the answer by removing unwanted 'A:' prefix if it exists
  if answer.startswith("A:"):
    answer = answer[2:].strip()  # Remove the "A:" part and any extra spaces

  print(f"Question:{question}")
  print(f"Answer:{answer}")
  print("*********************\n")

Question:What is a savings account?
Answer:Answer: A deposit account to save money and earn interest.
*********************

Question:What is a debit card?
Answer:Answer: A card used to spend directly from a linked bank account.
*********************

Question:What is a credit card?
Answer:Answer: A card that allows borrowing up to a pre-approved limit.
*********************

Question:What is mobile banking?
Answer:Answer: Banking services accessed through a smartphone app.
*********************

